# Bubble and Quick sort

## Common functions

In [21]:
from random import randint
from operator import itemgetter

In [22]:
idx = itemgetter(0)
val = itemgetter(1)

In [23]:
def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))

In [24]:
def is_sorted(arr):
    return not any(val(a) > val(b) for a, b in pairs(arr))

In [25]:
def move(arr, at, to):
    arr.insert(to, arr.pop(at))

## Bubble sort

In [27]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in pairs(arr):
            if val(a) > val(b):
                move(arr, idx(a), idx(b))

In [28]:
s = [randint(0, 100) for _ in range(25)]
print(s)
assert not is_sorted(s)
bubble_sort(s)
assert is_sorted(s)
print(s)

[5, 40, 67, 29, 43, 24, 17, 78, 88, 10, 53, 74, 20, 60, 45, 0, 63, 76, 29, 73, 37, 60, 93, 23, 58]
[0, 5, 10, 17, 20, 23, 24, 29, 29, 37, 40, 43, 45, 53, 58, 60, 60, 63, 67, 73, 74, 76, 78, 88, 93]


## Quick sort

In [29]:
def quick_sort(arr, start=0, end=-1):
    idx = start
    pivot_idx = end if end >= 0 else len(arr) - 1
    if idx < pivot_idx:
        pivot = arr[pivot_idx]
        while idx < pivot_idx:
            if arr[idx] > pivot:
                move(arr, idx, pivot_idx)                
                pivot_idx -= 1
            else:
                idx += 1
        quick_sort(arr, start, pivot_idx - 1)
        quick_sort(arr, pivot_idx + 1, end)

In [30]:
s = [randint(0, 100) for _ in range(25)]
print(s)
assert not is_sorted(s)
quick_sort(s)
assert is_sorted(s)
print(s)

[18, 3, 18, 70, 78, 49, 50, 18, 78, 73, 97, 2, 57, 82, 9, 37, 55, 96, 91, 98, 58, 67, 3, 47, 84]
[2, 3, 3, 9, 18, 18, 18, 37, 47, 49, 50, 55, 57, 58, 67, 70, 73, 78, 78, 82, 84, 91, 96, 97, 98]
